In [ ]:
import json
import itertools
from collections import defaultdict

In [ ]:
challenges = json.load(open("challenges.json"))

for c in challenges:
    c["champions"] = set(c["champions"])
    c["qte"] = int(c["qte"])
    
champions = set()
for c in challenges:
    champions |= c["champions"]
    
print(len(champions))

challenges_by_champions = defaultdict(list)
for challenge in challenges:
    for champion in challenge["champions"]:
        challenges_by_champions[champion].append(challenge["challenge_name"])
challenges_by_champions = dict(challenges_by_champions)

champions_by_challenge = {}
for challenge in challenges:
    champions_by_challenge[challenge["challenge_name"]] = challenge["champions"]

factions = {challenge["challenge_name"] for challenge in challenges if challenge["qte"] == 5}
synergies = {challenge["challenge_name"] for challenge in challenges if challenge["qte"] == 3}

print(factions)


In [ ]:
def find_challenges(comp):
    chall = set()
    for c in challenges:
        if len(c["champions"].intersection(comp)) >= c["qte"]:
            chall.add(c["challenge_name"])
    return chall


find_challenges(set(["Ahri", "Akali", "Shen", "Yasuo", "Zed"]))

In [ ]:
def find_comp(champions_, threshold=0):
    # sort them by number of challenges to find good match first
    champions_ = list(champions_)
    champions_.sort(key=lambda c: -len(challenges_by_champions[c]))
    
    # iterate over all champion combinations
    for comp in itertools.combinations(champions_, 5):
        comp_challenges = find_challenges(set(comp))
        if len(comp_challenges) >= threshold:
            yield comp, comp_challenges

In [ ]:
# find best comp by faction

for faction in factions:
    print(faction)
    c = champions_by_challenge[faction]
    comps = []
    for comp, comp_challenges in find_comp(c):
        l = comp, len(comp_challenges)
        comps.append(l)
    
    comps.sort(key=lambda l: -l[-1])
    
    best_comp_n = None
    for comp, n in comps:
        if best_comp_n is None:
            best_comp_n = n
            
        if n <= best_comp_n - 2:
            break
            
        print(n, comp)
        
    print()

In [ ]:
# find best comps overall

for comp, comp_challenges in find_comp(champions, 7):
    print(len(comp_challenges), comp)